#IMPORTS

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
#18H7Ta2mDeB2R2xeLrvZIlEliq3c8qTNa

In [ ]:
#@title uploader
DIR = "18H7Ta2mDeB2R2xeLrvZIlEliq3c8qTNa" #@param {type:"string"}
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# PyDrive reference:
# https://googledrive.github.io/PyDrive/docs/build/html/index.html


from google.colab import auth
auth.authenticate_user()

from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

# Replace the assignment below with your file ID
# to download a different file.
#
# A file ID looks like: 1gLBqEWEBQDYbKCDigHnUXNTkzl-OslSO

import io
from googleapiclient.http import MediaIoBaseDownload

request = drive_service.files().get_media(fileId=DIR)
downloaded = io.BytesIO()
downloader = MediaIoBaseDownload(downloaded, request)
done = False
while done is False:
  # _ is a placeholder for a progress object that we ignore.
  # (Our file is small, so we skip reporting progress.)
  _, done = downloader.next_chunk()

fileId = drive.CreateFile({'id': DIR }) #DRIVE_FILE_ID is file id example: 1iytA1n2z4go3uVCwE_vIKouTKyIDjEq
print(fileId['title'])  
fileId.GetContentFile(fileId['title'])  # Save Drive file as a local file

!unzip {fileId['title']}

In [ ]:
import cv2
import numpy as np
import os
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend as K
import keras
from keras.models import Sequential, Model,load_model
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping,ModelCheckpoint
from google.colab.patches import cv2_imshow
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,MaxPool2D
from keras.layers.core import Dropout
from keras.preprocessing import image
from keras.initializers import glorot_uniform

In [ ]:
tf.config.get_visible_devices('GPU')                #list_physical_devices('GPU') 

#PATHS


In [ ]:
train_path="/content/fruits-360/Training"
#val_path="/content/gdrive/MyDrive/bees/validate"
test_path="/content/fruits-360/Test"

#DATASETS PREPARATION

In [ ]:
train_datagen = ImageDataGenerator(zoom_range=0.15,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.15)
#val_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

In [ ]:
train_generator = train_datagen.flow_from_directory(train_path,target_size=(100, 100),batch_size=64,shuffle=True,class_mode='categorical')
#val_generator = test_datagen.flow_from_directory(val_path,target_size=(100, 100),batch_size=64,shuffle=False,class_mode='categorical')
test_generator = test_datagen.flow_from_directory(test_path,target_size=(100, 100),batch_size=64,shuffle=False,class_mode='categorical')

#MODEL

In [ ]:
model = Sequential()

model.add(Conv2D(100,(3,3),activation='relu', input_shape=(100, 100, 3), padding="same"))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(64,(3,3),activation='relu', padding="same"))
model.add(Conv2D(64,(3,3),activation='relu', padding="same"))

model.add(MaxPooling2D((2,2)))
model.add(Conv2D(128,(3,3),activation='relu', padding="same"))
model.add(Conv2D(128,(3,3),activation='relu', padding="same"))

model.add(MaxPooling2D((2,2)))
model.add(Conv2D(256,(3,3),activation='relu', padding="same"))
model.add(Conv2D(256,(3,3),activation='relu', padding="same"))
model.add(Conv2D(256,(3,3),activation='relu', padding="same"))

model.add(MaxPooling2D((2,2)))
model.add(Conv2D(512,(3,3),activation='relu', padding="same"))
model.add(Conv2D(512,(3,3),activation='relu', padding="same"))
model.add(Conv2D(512,(3,3),activation='relu', padding="same"))

model.add(MaxPooling2D((2,2)))
model.add(Conv2D(512,(3,3),activation='relu', padding="same"))
model.add(Conv2D(512,(3,3),activation='relu', padding="same"))
model.add(Conv2D(512,(3,3),activation='relu', padding="same"))

model.add(MaxPooling2D((2,2)))

#flatten
model.add(Flatten())


#hidden layers
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.5))

#output layers
model.add(Dense(131,activation='softmax'))#'sigmoid'

#MODELPROPS

In [ ]:
es=EarlyStopping(monitor='val_acc', mode='max', verbose=1, patience=20)

In [ ]:
mc = ModelCheckpoint('/content/gdrive/My Drive/Fruits/fruits_model_ckpt_vgg_19.h5', monitor='val_acc', mode='max')

model.compile(loss='categorical_crossentropy',#'hinge'
              optimizer='SGD',
              metrics=['acc'])


In [ ]:
H = model.fit(train_generator, validation_data=test_generator, epochs=100,verbose=1,callbacks=[mc, es])

In [ ]:
model.summary()

#MODEL SAVE

In [ ]:
model.save_weights("/content/gdrive/My Drive/Fruits/fruits_model_weights_vgg_19.h5")
model.save("/content/gdrive/MyDrive/Fruits/fruits_model_vgg_19.h5")

#PREDICTION/CONFUSION MATRIX

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report

test_loss, test_acc = model.evaluate(test_generator, steps=100)

print('test acc:', test_acc)

batch_size = 63

num_of_test_samples = 131

predictions = model.predict(test_generator,  num_of_test_samples // batch_size+1)

y_pred = np.argmax(predictions, axis=1)

true_classes = test_generator.classes

class_labels = list(test_generator.class_indices.keys())   

print(class_labels)

print(confusion_matrix(test_generator.classes, y_pred))

report = classification_report(true_classes, y_pred, target_names=class_labels, zero_division=1)

print(report)

In [ ]:
import matplotlib.pyplot as plt

#acc=H.history['acc']
val_acc=H.history['val_accuracy']
#loss=H.history['loss']
#val_loss=H.history['val_loss']

epochs=range(1,len(val_acc)+1)

#Train and test accuracy
#plt.plot(epochs,acc,'b',label='Training Accuracy',color="red")
plt.plot(epochs,val_acc,'b',label='Testing Accuracy')
plt.title("Train and test accuracy")
plt.legend()

plt.figure()

#Train and test loss
#plt.plot(epochs,loss,'b',label='Training loss',color="red")
#plt.plot(epochs,val_loss,'b',label='Testing loss')
#plt.title("Train and test loss")
#plt.legend()

plt.show()